In [1]:
import os
import shutil
from dotenv import load_dotenv
import autogen
import chromadb
from pathlib import Path
from autogen.agentchat.contrib.gpt_assistant_agent import GPTAssistantAgent
from autogen.retrieve_utils  import query_vector_db, create_vector_db_from_dir

load_dotenv()

True

In [2]:
# CREATING VECTOR STORE
class ChromaDBManager:
    def __init__(self, docs_path, db_path, collection_name):
        self.docs_path = Path(docs_path)
        self.db_path = Path(db_path)
        self.collection_name = collection_name

    def setup(self, clear_history=False):
        if not self.docs_path.exists():
            raise ValueError(f"The docs directory at {self.docs_path} does not exist.")
        if self.db_path.exists():
            if clear_history:
                shutil.rmtree(self.db_path)
                print(f"Removed existing directory: {self.db_path}")
                self.client = chromadb.PersistentClient(path=str(self.db_path))
                return
            # else load in from str(self.db_path)

       

    def create_vector_db(self, max_tokens=3000, chunk_mode="multi_lines"):
        create_vector_db_from_dir(
            dir_path=str(self.docs_path),
            client=self.client,
            collection_name=self.collection_name,
            max_tokens=max_tokens,
            chunk_mode=chunk_mode
        )
        print("Chroma DB created successfully.")

    def query_db(self, query_texts, n_results=10):
        return query_vector_db(
            query_texts=query_texts,
            n_results=n_results,
            client=self.client,
            collection_name=self.collection_name
        )
    
    def get_context(self, results):
        doc_contents = ""
        current_tokens = 0
        _doc_idx = self._doc_idx
        _tmp_retrieve_count = 0
        for idx, doc in enumerate(results["documents"][0]):
            if idx <= _doc_idx:
                continue
            if results["ids"][0][idx] in self._doc_ids:
                continue
            _doc_tokens = self.custom_token_count_function(doc, self._model)
            if _doc_tokens > self._context_max_tokens:
                func_print = f"Skip doc_id {results['ids'][0][idx]} as it is too long to fit in the context."
                print(func_print)
                self._doc_idx = idx
                continue
            if current_tokens + _doc_tokens > self._context_max_tokens:
                break
            func_print = f"Adding doc_id {results['ids'][0][idx]} to context."
            print(func_print)
            # print(func_print, "green"), flush=True)
            current_tokens += _doc_tokens
            doc_contents += doc + "\n"
            self._doc_idx = idx
            self._doc_ids.append(results["ids"][0][idx])
            self._doc_contents.append(doc)
            _tmp_retrieve_count += 1
            if _tmp_retrieve_count >= self.n_results:
                break
        return doc_contents


In [3]:
chroma_db_manager = ChromaDBManager('../data/docs', '../data/chromadb', 'autogen-discord')
chroma_db_manager.setup()
chroma_db_manager.create_vector_db()

ValueError: The docs directory at docs does not exist.

In [ ]:
# CONFIGURATIONS
config_list = autogen.config_list_from_dotenv(
    dotenv_file_path='../../.env',
    model_api_key_map={
        "gpt-4-1106-preview": "OPENAI_API_KEY",
    },
    filter_dict={
        "model": {
            "gpt-4-1106-preview",
        }
    }
)

query_vector_db_tool_config = {
    "name": "query_vector_db",
    "description": "Function to query the Chroma vector database.",
    "parameters": {
        "type": "object",
        "properties": {
            "query_texts": {"type": "array", "items": {"type": "string"}},
        },
        "required": ["query_texts"]
    }
}

get_context_tool_config = {
    "name": "get_context",
    "description": "Function to get the context from query results of the Chroma vector database.",
    "parameters": {
        "type": "object",
        "properties": {
            "results": {"type": "object"}
        },
        "required": ["results"]
    }
}


llm_config = {
    "config_list": config_list,
    "assistant_id": None,
    "tools": [
        {
            "type": "function",
            "function": query_vector_db_tool_config
        },
        {
            "type": "function",
            "function": get_context_tool_config
        }
    ],
    "model": "gpt-4-1106-preview"
}

In [ ]:
config_list

In [ ]:
RAG_ASSISTANT_SYSTEM_MESSAGE = """As a Retrieve-Augmented Chatbot, your role involves analyzing message history and using that to inform your responses. 
When a question is presented, you should:
Step 1: Determine the user's intent by considering the question and its context. The intent might involve generating code or answering a question, or both.
Step 2: Formulate a response based on the identified intent.

If the task involves code generation, adhere to the specified coding format. 
For question-answering tasks, aim for concise and precise responses.


For contextual understanding, use the `query_vector_db` tool to access information from the Chroma vector database. 
Enhance your replies by integrating context extracted from the Chroma DB using the `get_context` function. 
This approach will provide a well-informed basis for your responses.
Say TERMINATE when everything is done and the overall task is complete.
"""

In [ ]:
# INSTANTIATE AGENTS

sql_assistant = GPTAssistantAgent(
    name="Chroma_RAG_Assistant",
    instructions=RAG_ASSISTANT_SYSTEM_MESSAGE,
    llm_config=llm_config
)

sql_assistant.register_function(
    function_map={
        "query_vector_db": chroma_db_manager.query_db,
        "get_context": chroma_db_manager.get_context   
    }
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
    code_execution_config=False,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3
)

initial_message = "What is Autogen?"
user_proxy.initiate_chat(sql_assistant, message=initial_message)

In [ ]:
user_proxy.send(recipient=sql_assistant, message="Explain what the config_list is?")

In [ ]:
user_proxy.send(recipient=sql_assistant, message="How can you use Chainlit with Autogen" )